In [0]:
from pyspark.sql import SparkSession, functions as f

df_cpt_codes=spark.read.parquet('/mnt/bronze/cpt_codes/*.parquet')

df_cpt_codes.createOrReplaceTempView('cpt_codes')

In [0]:
%sql
DESCRIBE cpt_codes;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW quality_checks AS
SELECT 
  cpt_codes,
  procedure_code_category,
  procedure_code_descriptions,
  code_status,
  CASE
    WHEN cpt_codes IS NULL OR procedure_code_descriptions IS NULL THEN TRUE
    ELSE FALSE
  END AS is_quarantined FROM cpt_codes; 

In [0]:
%sql
CREATE TABLE IF NOT EXISTS healthcarerevenuecyclemanagement_databricks.silver.cpt_codes(
  cpt_codes STRING,
  procedure_code_category STRING,
  procedure_code_descriptions STRING,
  code_status STRING,
  audit_inserteddate TIMESTAMP,
  audit_updateddate TIMESTAMP,
  is_quarantined BOOLEAN,
  is_current BOOLEAN
) USING DELTA;

In [0]:
%sql
MERGE INTO healthcarerevenuecyclemanagement_databricks.silver.cpt_codes AS target
USING quality_checks AS source
ON target.cpt_codes = source.cpt_codes
AND is_current = TRUE
WHEN MATCHED AND (
  target.procedure_code_category <> source.procedure_code_category OR
  target.procedure_code_descriptions <> source.procedure_code_descriptions OR
  target.code_status <> source.code_status OR
  target.is_quarantined <> source.is_quarantined
) THEN UPDATE SET is_current = FALSE, audit_updateddate = current_timestamp();

In [0]:
%sql
MERGE INTO healthcarerevenuecyclemanagement_databricks.silver.cpt_codes AS target
USING quality_checks AS source
ON target.cpt_codes = source.cpt_codes
AND is_current = TRUE
WHEN NOT MATCHED THEN INSERT (
  cpt_codes,
  procedure_code_category,
  procedure_code_descriptions,
  code_status,
  audit_inserteddate,
  audit_updateddate,
  is_quarantined,
  is_current
) VALUES (
  source.cpt_codes,
  source.procedure_code_category,
  source.procedure_code_descriptions,
  source.code_status,
  current_timestamp(),
  current_timestamp(),
  source.is_quarantined,
  TRUE
  );

In [0]:
%sql
SELECT * FROM healthcarerevenuecyclemanagement_databricks.silver.cpt_codes;